# GCP Bucket Crawler and Catalog Generator

This notebook crawls a GCP storage bucket to discover vector and raster data, then generates collections and a comprehensive catalog.

## Prerequisites

1. Install required packages:
   ```bash
   pip install google-cloud-storage
   ```

2. Set up authentication:
   - Service account: `export GOOGLE_APPLICATION_CREDENTIALS='path/to/key.json'`
   - Or use: `gcloud auth application-default login`

In [ ]:
# Import required libraries
import json
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Optional
from urllib.parse import urljoin
import os
from google.cloud import storage

## Configuration

Update these values for your specific bucket:

In [ ]:
# Configuration - update these values for your specific bucket
bucket_name = "swhm_data"  # Just the bucket name, not the full URL
prefix = "public/layers/"  # Path prefix within the bucket
project_id = None  # Set your GCP project ID if needed

print(f"Bucket: {bucket_name}")
print(f"Prefix: {prefix}")
print(f"Project ID: {project_id}")

## GCP Bucket Crawler Class

This class handles the connection to GCP and crawling of the bucket:

In [ ]:
class GCPBucketCrawler:
    def __init__(self, bucket_name: str, prefix: str = "", project_id: Optional[str] = None):
        """
        Initialize the crawler with GCP bucket details.
        
        Args:
            bucket_name: Name of the GCP storage bucket (e.g., 'swhm_data')
            prefix: Prefix to filter objects (e.g., 'public/layers/')
            project_id: GCP project ID (optional, will use default if not provided)
        """
        self.bucket_name = bucket_name
        self.prefix = prefix
        self.project_id = project_id
        self.vectors = []
        self.rasters = []
        
        # Initialize the GCS client
        try:
            if project_id:
                self.client = storage.Client(project=project_id)
            else:
                self.client = storage.Client()
            self.bucket = self.client.bucket(bucket_name)
            print(f"Successfully connected to bucket: {bucket_name}")
        except Exception as e:
            print(f"Error initializing GCS client: {e}")
            print("Make sure you have proper authentication set up:")
            print("1. Set GOOGLE_APPLICATION_CREDENTIALS environment variable")
            print("2. Or run 'gcloud auth application-default login'")
            self.client = None
            self.bucket = None
        
    def crawl_bucket(self) -> Dict:
        """
        Crawl the GCP bucket to discover all vectors and rasters.
        Returns a dictionary with discovered items.
        """
        if not self.client or not self.bucket:
            print("No valid GCS client available, creating sample data...")
            return self._create_sample_data()
            
        print(f"Crawling bucket '{self.bucket_name}' with prefix '{self.prefix}'...")
        
        try:
            # List all blobs in the bucket with the specified prefix
            blobs = self.bucket.list_blobs(prefix=self.prefix)
            
            blob_count = 0
            for blob in blobs:
                blob_count += 1
                self._process_blob(blob)
                
            print(f"Processed {blob_count} objects from bucket")
            
        except Exception as e:
            print(f"Error crawling bucket: {e}")
            return self._create_sample_data()
            
        return {
            'vectors': self.vectors,
            'rasters': self.rasters,
            'total_items': len(self.vectors) + len(self.rasters)
        }
    
    def _process_blob(self, blob):
        """Process a single blob to determine if it's a vector or raster."""
        blob_name = blob.name
        blob_path = Path(blob_name)
        
        # Skip directories (blobs ending with '/')
        if blob_name.endswith('/'):
            return
            
        # Check for vector files
        if 'vectors/' in blob_name and blob_path.suffix.lower() in ['.geojson', '.json']:
            self._add_vector_item(blob)
            
        # Check for raster files  
        elif 'rasters/' in blob_name and blob_path.suffix.lower() in ['.tiff', '.tif', '.gtiff']:
            self._add_raster_item(blob)
    
    def _add_vector_item(self, blob):
        """Add a vector item to the collection."""
        blob_path = Path(blob.name)
        item_name = blob_path.stem
        
        # Create public URL
        public_url = f"https://storage.googleapis.com/{self.bucket_name}/{blob.name}"
        
        vector_item = {
            'name': item_name,
            'filename': blob.name,
            'url': public_url,
            'type': 'vector',
            'format': 'GeoJSON',
            'size_bytes': blob.size,
            'content_type': blob.content_type,
            'created': blob.time_created.isoformat() if blob.time_created else None,
            'updated': blob.updated.isoformat() if blob.updated else None,
            'discovered_at': datetime.now().isoformat(),
            'etag': blob.etag,
            'md5_hash': blob.md5_hash
        }
        
        self.vectors.append(vector_item)
        print(f"Found vector: {item_name}")
    
    def _add_raster_item(self, blob):
        """Add a raster item to the collection."""
        blob_path = Path(blob.name)
        item_name = blob_path.stem
        
        # Create public URL
        public_url = f"https://storage.googleapis.com/{self.bucket_name}/{blob.name}"
        
        raster_item = {
            'name': item_name,
            'filename': blob.name,
            'url': public_url,
            'type': 'raster',
            'format': 'GeoTIFF',
            'size_bytes': blob.size,
            'content_type': blob.content_type,
            'created': blob.time_created.isoformat() if blob.time_created else None,
            'updated': blob.updated.isoformat() if blob.updated else None,
            'discovered_at': datetime.now().isoformat(),
            'etag': blob.etag,
            'md5_hash': blob.md5_hash
        }
        
        self.rasters.append(raster_item)
        print(f"Found raster: {item_name}")
    
    def get_blob_info(self, blob_name: str) -> Optional[Dict]:
        """Get detailed information about a specific blob."""
        if not self.bucket:
            return None
            
        try:
            blob = self.bucket.blob(blob_name)
            if blob.exists():
                return {
                    'name': blob.name,
                    'size': blob.size,
                    'content_type': blob.content_type,
                    'created': blob.time_created.isoformat() if blob.time_created else None,
                    'updated': blob.updated.isoformat() if blob.updated else None,
                    'etag': blob.etag,
                    'md5_hash': blob.md5_hash,
                    'public_url': f"https://storage.googleapis.com/{self.bucket_name}/{blob.name}"
                }
        except Exception as e:
            print(f"Error getting blob info for {blob_name}: {e}")
            
        return None
    
    def _create_sample_data(self):
        """Create sample data structure when bucket can't be crawled directly."""
        print("Creating sample data structure...")
        
        base_url = f"https://storage.googleapis.com/{self.bucket_name}"
        
        # Sample vectors based on your structure
        sample_vectors = [
            {
                'name': 'vector1',
                'filename': f'{self.prefix}vectors/vector1/vector1.geojson',
                'url': f"{base_url}/{self.prefix}vectors/vector1/vector1.geojson",
                'type': 'vector',
                'format': 'GeoJSON',
                'size_bytes': None,
                'content_type': 'application/geo+json',
                'created': None,
                'updated': None,
                'discovered_at': datetime.now().isoformat(),
                'etag': None,
                'md5_hash': None
            }
        ]
        
        # Sample rasters based on your structure
        sample_rasters = [
            {
                'name': 'raster1',
                'filename': f'{self.prefix}rasters/raster1/raster1.tiff',
                'url': f"{base_url}/{self.prefix}rasters/raster1/raster1.tiff",
                'type': 'raster',
                'format': 'GeoTIFF',
                'size_bytes': None,
                'content_type': 'image/tiff',
                'created': None,
                'updated': None,
                'discovered_at': datetime.now().isoformat(),
                'etag': None,
                'md5_hash': None
            }
        ]
        
        self.vectors = sample_vectors
        self.rasters = sample_rasters
        
        return {
            'vectors': self.vectors,
            'rasters': self.rasters,
            'total_items': len(self.vectors) + len(self.rasters)
        }

## Catalog Generator Class

This class generates STAC-compliant collections and catalogs:

In [ ]:
class CatalogGenerator:
    def __init__(self, crawler_data: Dict):
        """Initialize with data from the crawler."""
        self.data = crawler_data
        
    def generate_vector_collection(self) -> Dict:
        """Generate a vector collection."""
        collection = {
            "type": "Collection",
            "id": "swhm-vectors",
            "title": "SWHM Vector Collection",
            "description": "Collection of vector datasets from SWHM data bucket",
            "keywords": ["vectors", "geojson", "swhm"],
            "license": "proprietary",
            "extent": {
                "spatial": {
                    "bbox": [[-180, -90, 180, 90]]  # Global bbox - update with actual bounds
                },
                "temporal": {
                    "interval": [[None, None]]
                }
            },
            "providers": [
                {
                    "name": "SWHM Data",
                    "roles": ["producer", "processor", "host"],
                    "url": "https://storage.googleapis.com/swhm_data/"
                }
            ],
            "items": []
        }
        
        # Add vector items
        for vector in self.data['vectors']:
            item = {
                "type": "Feature",
                "id": vector['name'],
                "properties": {
                    "title": vector['name'].replace('_', ' ').title(),
                    "description": f"Vector dataset: {vector['name']}",
                    "created": vector['discovered_at'],
                    "updated": vector['discovered_at']
                },
                "geometry": None,  # Would need to extract from actual GeoJSON
                "bbox": None,  # Would need to calculate from geometry
                "assets": {
                    "data": {
                        "href": vector['url'],
                        "type": "application/geo+json",
                        "title": "GeoJSON data",
                        "roles": ["data"],
                        "file:size": vector.get('size_bytes'),
                        "file:checksum": vector.get('md5_hash')
                    }
                }
            }
            collection["items"].append(item)
            
        return collection
    
    def generate_raster_collection(self) -> Dict:
        """Generate a raster collection."""
        collection = {
            "type": "Collection",
            "id": "swhm-rasters",
            "title": "SWHM Raster Collection",
            "description": "Collection of raster datasets from SWHM data bucket",
            "keywords": ["rasters", "geotiff", "swhm"],
            "license": "proprietary",
            "extent": {
                "spatial": {
                    "bbox": [[-180, -90, 180, 90]]  # Global bbox - update with actual bounds
                },
                "temporal": {
                    "interval": [[None, None]]
                }
            },
            "providers": [
                {
                    "name": "SWHM Data",
                    "roles": ["producer", "processor", "host"],
                    "url": "https://storage.googleapis.com/swhm_data/"
                }
            ],
            "items": []
        }
        
        # Add raster items
        for raster in self.data['rasters']:
            item = {
                "type": "Feature",
                "id": raster['name'],
                "properties": {
                    "title": raster['name'].replace('_', ' ').title(),
                    "description": f"Raster dataset: {raster['name']}",
                    "created": raster['discovered_at'],
                    "updated": raster['discovered_at']
                },
                "geometry": None,  # Would need to extract from raster metadata
                "bbox": None,  # Would need to calculate from raster bounds
                "assets": {
                    "data": {
                        "href": raster['url'],
                        "type": "image/tiff; application=geotiff",
                        "title": "GeoTIFF data",
                        "roles": ["data"],
                        "file:size": raster.get('size_bytes'),
                        "file:checksum": raster.get('md5_hash')
                    }
                }
            }
            collection["items"].append(item)
            
        return collection
    
    def generate_master_catalog(self, vector_collection: Dict, raster_collection: Dict) -> Dict:
        """Generate the master catalog containing all collections."""
        catalog = {
            "type": "Catalog",
            "id": "swhm-data-catalog",
            "title": "SWHM Data Catalog",
            "description": "Master catalog for SWHM vector and raster datasets",
            "stac_version": "1.0.0",
            "created": datetime.now().isoformat(),
            "updated": datetime.now().isoformat(),
            "keywords": ["swhm", "vectors", "rasters", "geospatial"],
            "providers": [
                {
                    "name": "SWHM Data",
                    "roles": ["producer", "processor", "host"],
                    "url": "https://storage.googleapis.com/swhm_data/"
                }
            ],
            "links": [
                {
                    "rel": "self",
                    "href": "./catalog.json",
                    "type": "application/json",
                    "title": "SWHM Data Catalog"
                },
                {
                    "rel": "child",
                    "href": "./vectors/collection.json",
                    "type": "application/json",
                    "title": "Vector Collection"
                },
                {
                    "rel": "child",
                    "href": "./rasters/collection.json",
                    "type": "application/json",
                    "title": "Raster Collection"
                }
            ],
            "collections": [
                {
                    "id": vector_collection['id'],
                    "title": vector_collection['title'],
                    "description": vector_collection['description'],
                    "href": "./vectors/collection.json"
                },
                {
                    "id": raster_collection['id'],
                    "title": raster_collection['title'],
                    "description": raster_collection['description'],
                    "href": "./rasters/collection.json"
                }
            ],
            "summaries": {
                "total_items": self.data['total_items'],
                "vector_items": len(self.data['vectors']),
                "raster_items": len(self.data['rasters']),
                "formats": ["GeoJSON", "GeoTIFF"]
            }
        }
        
        return catalog

## Utility Functions

Helper function to save JSON files:

In [ ]:
def save_json(data: Dict, filepath: str):
    """Save data to JSON file with pretty formatting."""
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"Saved: {filepath}")

## Initialize Crawler

Create the crawler instance and connect to the GCP bucket:

In [ ]:
# Initialize crawler
crawler = GCPBucketCrawler(bucket_name, prefix, project_id)

## Crawl the Bucket

Start the bucket crawling process to discover vector and raster data:

In [ ]:
# Crawl the bucket
print("Starting bucket crawl...")
crawl_data = crawler.crawl_bucket()

print(f"Found {len(crawl_data['vectors'])} vectors and {len(crawl_data['rasters'])} rasters")

## Display Results

Show a summary of the discovered data:

In [ ]:
# Display discovered vectors
print("\n=== DISCOVERED VECTORS ===")
for i, vector in enumerate(crawl_data['vectors'], 1):
    print(f"{i}. {vector['name']} ({vector['format']})")
    print(f"   URL: {vector['url']}")
    if vector['size_bytes']:
        print(f"   Size: {vector['size_bytes']:,} bytes")
    print()

# Display discovered rasters
print("\n=== DISCOVERED RASTERS ===")
for i, raster in enumerate(crawl_data['rasters'], 1):
    print(f"{i}. {raster['name']} ({raster['format']})")
    print(f"   URL: {raster['url']}")
    if raster['size_bytes']:
        print(f"   Size: {raster['size_bytes']:,} bytes")
    print()

## Generate Collections and Catalog

Create the STAC collections and master catalog:

In [ ]:
# Generate collections and catalog
generator = CatalogGenerator(crawl_data)

# Generate collections
print("Generating vector collection...")
vector_collection = generator.generate_vector_collection()

print("Generating raster collection...")
raster_collection = generator.generate_raster_collection()

# Generate master catalog
print("Generating master catalog...")
master_catalog = generator.generate_master_catalog(vector_collection, raster_collection)

print("\n✅ All collections and catalog generated successfully!")

## Preview Generated Catalog

Display a preview of the generated master catalog:

In [ ]:
# Display catalog summary
print("=== MASTER CATALOG SUMMARY ===")
print(f"Title: {master_catalog['title']}")
print(f"Description: {master_catalog['description']}")
print(f"STAC Version: {master_catalog['stac_version']}")
print(f"Total Items: {master_catalog['summaries']['total_items']}")
print(f"Vector Items: {master_catalog['summaries']['vector_items']}")
print(f"Raster Items: {master_catalog['summaries']['raster_items']}")
print(f"Formats: {', '.join(master_catalog['summaries']['formats'])}")

print("\n=== COLLECTIONS ===")
for collection in master_catalog['collections']:
    print(f"- {collection['title']}: {collection['description']}")

## Save Catalog Files

Save all generated collections and catalogs to JSON files:

In [ ]:
# Save all files
print("Saving catalog files...")
save_json(master_catalog, "catalog/catalog.json")
save_json(vector_collection, "catalog/vectors/collection.json")
save_json(raster_collection, "catalog/rasters/collection.json")

# Save summary with enhanced metadata
summary = {
    "crawl_summary": {
        "bucket_name": bucket_name,
        "prefix": prefix,
        "crawl_time": datetime.now().isoformat(),
        "total_items": crawl_data['total_items'],
        "vectors_found": len(crawl_data['vectors']),
        "rasters_found": len(crawl_data['rasters'])
    },
    "discovered_vectors": crawl_data['vectors'],
    "discovered_rasters": crawl_data['rasters']
}
save_json(summary, "catalog/crawl_summary.json")

print(f"\n✅ Catalog generation complete!")
print(f"   - Master catalog: catalog/catalog.json")
print(f"   - Vector collection: catalog/vectors/collection.json")
print(f"   - Raster collection: catalog/rasters/collection.json")
print(f"   - Crawl summary: catalog/crawl_summary.json")

## Explore Individual Blob Information

Get detailed information about specific blobs in the bucket:

In [ ]:
# Example: Get information about a specific blob
# Uncomment and modify the blob name as needed

# blob_name = "public/layers/vectors/example/example.geojson"
# blob_info = crawler.get_blob_info(blob_name)
# 
# if blob_info:
#     print(f"\n=== BLOB INFO: {blob_name} ===")
#     for key, value in blob_info.items():
#         print(f"{key}: {value}")
# else:
#     print(f"Blob not found or error accessing: {blob_name}")

print("\n💡 To get info about a specific blob, uncomment and modify the code above")

## Export as Python Script

If you want to run this as a standalone script, here's the complete code:

In [ ]:
# Create a standalone Python script
script_content = '''#!/usr/bin/env python3
"""
GCP Bucket Crawler and Catalog Generator
Generated from Jupyter notebook
"""

# This script combines all the classes and functions from above
# You can copy the individual cells above to create a complete script

def main():
    """Main function to run the crawler."""
    # Configuration
    bucket_name = "swhm_data"
    prefix = "public/layers/"
    project_id = None
    
    # Initialize and run crawler
    crawler = GCPBucketCrawler(bucket_name, prefix, project_id)
    crawl_data = crawler.crawl_bucket()
    
    # Generate catalog
    generator = CatalogGenerator(crawl_data)
    vector_collection = generator.generate_vector_collection()
    raster_collection = generator.generate_raster_collection()
    master_catalog = generator.generate_master_catalog(vector_collection, raster_collection)
    
    # Save files
    save_json(master_catalog, "catalog/catalog.json")
    save_json(vector_collection, "catalog/vectors/collection.json")
    save_json(raster_collection, "catalog/rasters/collection.json")
    
    print("✅ Catalog generation complete!")

if __name__ == "__main__":
    main()
'''

# Save the script
with open('gcp_bucket_crawler.py', 'w') as f:
    # Note: In a real scenario, you'd include all the class definitions here
    f.write(script_content)
    
print("📄 Standalone script template saved as 'gcp_bucket_crawler.py'")
print("   Note: You'll need to copy the class definitions from the cells above")

## Troubleshooting

Common issues and solutions:

In [ ]:
# Check authentication status
print("=== TROUBLESHOOTING GUIDE ===")
print("\n1. Authentication Issues:")
print("   - Set GOOGLE_APPLICATION_CREDENTIALS environment variable")
print("   - Or run: gcloud auth application-default login")
print("   - Ensure service account has Storage Object Viewer permissions")

print("\n2. Package Installation:")
print("   - pip install google-cloud-storage")

print("\n3. Bucket Access:")
print("   - Verify bucket name is correct")
print("   - Check if bucket is publicly accessible or requires authentication")
print("   - Verify prefix path exists in the bucket")

print("\n4. File Detection:")
print("   - Vector files should be in 'vectors/' subdirectory with .geojson/.json extension")
print("   - Raster files should be in 'rasters/' subdirectory with .tiff/.tif/.gtiff extension")

# Check current environment
print("\n=== CURRENT ENVIRONMENT ===")
import sys
print(f"Python version: {sys.version}")

try:
    import google.cloud.storage
    print(f"google-cloud-storage: Available")
except ImportError:
    print(f"google-cloud-storage: NOT INSTALLED")
    print("   Run: pip install google-cloud-storage")

# Check environment variables
gcp_creds = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')
if gcp_creds:
    print(f"GOOGLE_APPLICATION_CREDENTIALS: {gcp_creds}")
else:
    print("GOOGLE_APPLICATION_CREDENTIALS: Not set")
    print("   Consider setting this environment variable")